In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import tensorflow
import keras

Using TensorFlow backend.


In [2]:
import struct

class WavFileHelper():
    
    def read_file_properties(self, filename):

        wave_file = open(filename,"rb")
        
        riff = wave_file.read(12)
        fmt = wave_file.read(36)
        
        num_channels_string = fmt[10:12]
        num_channels = struct.unpack('<H', num_channels_string)[0]

        sample_rate_string = fmt[12:16]
        sample_rate = struct.unpack("<I",sample_rate_string)[0]
        
        bit_depth_string = fmt[22:24]
        bit_depth = struct.unpack("<H",bit_depth_string)[0]

        return (num_channels, sample_rate, bit_depth)
wavfilehelper = WavFileHelper() 

In [3]:
audiodata = []
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'G:/urban_dataset'

metadata = pd.read_csv(fulldatasetpath + '/metadata/UrbanSound8K.csv')


for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath('G:/urban_dataset/'),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    data = wavfilehelper.read_file_properties(file_name)
    audiodata.append(data)

# Convert into a Panda dataframe
audiodf = pd.DataFrame(audiodata, columns=['num_channels','sample_rate','bit_depth'])

In [4]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') # default is Kaiser_best
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(audio),
        sr=sample_rate).T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return tonnetz

In [5]:
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'G:/urban_dataset'

metadata = pd.read_csv(fulldatasetpath + '/metadata/UrbanSound8K.csv')

features = []
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

C:\Users\Admin\Anaconda3\envs\tensorflow\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


Finished feature extraction from  8732  files


In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [7]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

In [8]:
# neural network dimensions
n_dim =x_train.shape[1]
n_classes = y_train.shape[1]
n_hidden_units_1 = n_dim
n_hidden_units_2 = 400 # approx n_dim * 2
n_hidden_units_3 = 200 # half of layer 2

In [9]:
print(n_dim)
print("Features: {} Classes: {} ".format(n_dim, n_classes))

6
Features: 6 Classes: 10 


In [10]:
def create_model(activation_function='relu',init_type='normal', kernel_initializer="uniform", optimiser='Adamax', dropout_rate=0.5):
    model = Sequential()
    # layer 1
    #model.add(Convolution2D(n_hidden_units_1, input_dim=n_dim, activation=activation_function,kernel_initializer="uniform"))
    #model.add(MaxPooling2D(2,2))
    model.add(Dense(n_hidden_units_1, input_dim=n_dim, init=init_type, activation=activation_function))
    # layer 2
    model.add(Dense(n_hidden_units_2, kernel_initializer="uniform", activation=activation_function))
    model.add(Dropout(dropout_rate))
    # layer 3
    model.add(Dense(n_hidden_units_3, kernel_initializer="uniform" , activation=activation_function))
    model.add(Dropout(dropout_rate))
    # output layer
    model.add(Dense(n_classes, kernel_initializer="uniform", activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=optimiser, metrics=['accuracy'])
    return model

In [11]:
model = create_model()
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')

C:\Users\Admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, input_dim=6, activation="relu", kernel_initializer="normal")`
  


In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 400)               2800      
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total params: 85,052
Trainable params: 85,052
Non-trainable params: 0
__________________________________________________

In [13]:
history = model.fit(x_train, y_train, epochs=30, validation_split=0.1, callbacks=[earlystop], batch_size=24)

Train on 6286 samples, validate on 699 samples
Epoch 1/30
6286/6286 [==============================] - 3s 406us/step - loss: 2.2709 - acc: 0.1147 - val_loss: 2.2497 - val_acc: 0.0987
Epoch 2/30
6286/6286 [==============================] - 2s 355us/step - loss: 2.2634 - acc: 0.1110 - val_loss: 2.2493 - val_acc: 0.1102
Epoch 3/30
6286/6286 [==============================] - 2s 361us/step - loss: 2.2630 - acc: 0.1115 - val_loss: 2.2500 - val_acc: 0.1116
Epoch 4/30
6286/6286 [==============================] - 3s 403us/step - loss: 2.2608 - acc: 0.1147 - val_loss: 2.2470 - val_acc: 0.1102
Epoch 5/30
6286/6286 [==============================] - 2s 345us/step - loss: 2.2574 - acc: 0.1153 - val_loss: 2.2402 - val_acc: 0.1159
Epoch 6/30
6286/6286 [==============================] - 2s 353us/step - loss: 2.2538 - acc: 0.1187 - val_loss: 2.2383 - val_acc: 0.1102
Epoch 7/30
6286/6286 [==============================] - 2s 353us/step - loss: 2.2456 - acc: 0.1290 - val_loss: 2.2274 - val_acc: 0.1102
E

In [14]:
score, accuracy = model.evaluate(x = x_test, y = y_test , verbose = 1)
print("ACCURACY:",(accuracy*100))
print("SCORE:",score)

1747/1747 [==============================] - 0s 107us/step
ACCURACY: 16.94333142700643
SCORE: 2.1650471442347605


In [15]:
model.save("nn_tonnetz_model.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('nn_tonnetz_model.h5')
# summarize model.
model.summary()
# load dataset

# evaluate the model
score = model.evaluate(x=x_test,y=y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 400)               2800      
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total params: 85,052
Trainable params: 85,052
Non-trainable params: 0
__________________________________________________